<a href="https://colab.research.google.com/github/DavidUpegui/Home_Loan_approval_ML/blob/main/Evaluaci%C3%B3n_inteligente_de_cr%C3%A9ditos_hipotecarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sé mi aprobación hipotecaria: Evaluación inteligente de créditos hipotecarios